<a href="https://colab.research.google.com/github/sauravjoshi/NLP/blob/master/music_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/NLP'

In [0]:
!pip install pydub

In [0]:
import numpy as np
import pandas as pd
import pydub

from keras.layers import LSTM, Dense, LeakyReLU
from keras.models import Sequential, load_model
from scipy.io.wavfile import read, write

#from keras.layers import CuDNNLSTM as LSTM

In [9]:
!ls gdrive/My\ Drive/NLP/

glove.6B.100d.txt  glove.6B.50d.txt  Numb_piano.mp3  robert_frost.txt


In [15]:
sound = pydub.AudioSegment.from_mp3('./gdrive/My Drive/NLP/Numb_piano.mp3')
sound.export('./gdrive/My Drive/NLP/Numb.wav', format='wav')
sound = pydub.AudioSegment.from_mp3('./gdrive/My Drive/NLP/Eminem.mp3')
sound.export('./gdrive/My Drive/NLP/Eminem.wav', format='wav')

<_io.BufferedRandom name='./gdrive/My Drive/NLP/Eminem1.wav'>

In [0]:
rate, music1 = read('./gdrive/My Drive/NLP/Numb.wav')
rate, music2 = read('./gdrive/My Drive/NLP/Eminem.wav')

In [0]:
music1 = pd.DataFrame(music1[0:400000, :])
music2 = pd.DataFrame(music2[0:400000, :])

In [0]:
def create_train_dataset(df, look_back, train=True):
    dataX1, dataX2 , dataY1 , dataY2 = [],[],[],[]
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        if train:
            dataY1.append(df.iloc[i + look_back, 0])
            dataY2.append(df.iloc[i + look_back, 1])
    if train:
        return np.array(dataX1), np.array(dataX2), np.array(dataY1), np.array(dataY2)
    else:
        return np.array(dataX1), np.array(dataX2)

In [0]:
X1, X2, y1, y2 = create_train_dataset(
    pd.concat([music1.iloc[0:160000, :],music2.iloc[0:160000, :]], axis=0),
    look_back=3, train=True)

In [0]:
test1, test2 = create_train_dataset(
    pd.concat([music1.iloc[160001 : 400000, :],music2.iloc[160001 : 400000, :]], axis=0),
    look_back=3, train=False)

In [27]:
print(X1.shape)
print(X2.shape)
print(y1.shape)
print(y2.shape)

(319996, 3)
(319996, 3)
(319996,)
(319996,)


In [0]:
X1 = X1.reshape((-1, 1, 3))
X2 = X2.reshape((-1, 1, 3))

In [0]:
rnn1 = Sequential()
rnn1.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=50, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=25, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=12, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=1, activation='linear'))
rnn1.add(LeakyReLU())

rnn1.compile(optimizer='adam', loss='mean_squared_error')
rnn1.fit(X1, y1, epochs=20, batch_size=100)

In [0]:
rnn2 = Sequential()
rnn2.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=50, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=25, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=12, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=1, activation='linear'))
rnn2.add(LeakyReLU())

rnn2.compile(optimizer='adam', loss='mean_squared_error')
rnn2.fit(X2, y2, epochs=20, batch_size=100)

In [0]:
rnn1.model.save('./gdrive/My Drive/NLP/music_generation_rnn1.h5')
rnn2.model.save('./gdrive/My Drive/NLP/music_generation_rnn2.h5')

In [0]:
# loading the saved models
rnn1 = load_model('./gdrive/My Drive/NLP/music_generation_rnn1.h5')
rnn2 = load_model('./gdrive/My Drive/NLP/music_generation_rnn2.h5')

In [0]:
pred_rnn1 = rnn1.predict(test1.reshape(-1, 1, 3))
pred_rnn2 = rnn2.predict(test2.reshape(-1, 1, 3))

In [0]:
write('./gdrive/My Drive/NLP/pred_rnn.wav', rate,
      pd.concat([pd.DataFrame(pred_rnn1.astype('int16')),
                 pd.DataFrame(pred_rnn2.astype('int16'))],
                 axis=1).values)

In [0]:
write('./gdrive/My Drive/NLP/original.wav',rate,
      pd.concat([music1.iloc[160001 : 400000, :], music2.iloc[160001 : 400000, :]],
                axis=0).values)